In [7]:
!pip install openai==1.26.0
!pip show openai

   ---------------------------------------- 0.0/314.1 kB ? eta -:--:--
   ----- --------------------------------- 41.0/314.1 kB 653.6 kB/s eta 0:00:01
   -------------------------- ------------- 204.8/314.1 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 314.1/314.1 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.27.3
    Uninstalling openai-0.27.3:
      Successfully uninstalled openai-0.27.3
Name: openai
Version: 1.26.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: C:\Users\jason\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [31]:
import openai

client = openai.OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "THIS MUST BE 127.0.0.1, not localhost
    api_key = "sk-no-key-required"
)

def stream_response(question):
    system_message = ""
    completion = client.chat.completions.create(
        model="llama-3",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question}
        ],
        stream=True,  # Whether to stream the response or return the full response at once
        max_tokens=8000,  # The maximum number of tokens to generate in the completion
        temperature=0.5,  # The temperature of the model, controlling the randomness of the output
        top_p=1.0,  # The nucleus sampling parameter, controlling the diversity of the output
        n=1,  # The number of completions to generate
        stop=None,  # A list of stop sequences to stop the completion at
        presence_penalty=0.0,  # The penalty for generating tokens that appear in the prompt
        frequency_penalty=0.0,  # The penalty for generating tokens that appear frequently in the prompt
        logit_bias={},  # A dictionary of token IDs and associated biases to apply to the logits
    )
    
    # print(completion.choices[0].message)
    for chunk in completion:
        print(chunk.choices[0].delta.content, end="")

In [1]:
# import openai
# 
# openai.api_key = ""
# openai.api_base = "http://localhost:8080/v1"
# # openai.api_base = "http://127.0.0.1:8080/v1"
# 
# def stream_response(question):
#     system_message = "You are ChatGPT, an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests."
# 
#     response = openai.ChatCompletion.create(
#         model="llama-3",
#         messages=[
#             {"role": "system", "content": system_message},
#             {"role": "user", "content": question}
#         ],
#         stream=True,
#         temperature=0.5,
#         top_p=1.0,
#         n=1,
#         stop=None,
#         presence_penalty=0.0,
#         frequency_penalty=0.0,
#         logit_bias={}
#     )
# 
#     for chunk in response:
#         print(chunk.choices[0].delta.content, end="")

In [15]:
stream_response(f"""
Who are some key buddhist figures in history?  What did they contribute?
""")

Here are some key Buddhist figures in history and their contributions:

1. **Siddhartha Gautama (563 BCE - 483 BCE)**: Also known as the Buddha, he is the founder of Buddhism. He was a prince who renounced his royal life and achieved enlightenment through meditation and self-reflection. His teachings, known as the Four Noble Truths, form the foundation of Buddhism.

Contribution: Established the core teachings of Buddhism, including the Four Noble Truths and the Eightfold Path.

2. **Ashoka (269-232 BCE)**: An Indian emperor who converted to Buddhism and spread it throughout his kingdom. He was instrumental in the development of Buddhist institutions and the spread of Buddhism to Sri Lanka and other parts of Asia.

Contribution: Established Buddhism as a state religion, built many Buddhist monuments, and sent missionaries to spread Buddhism.

3. **Nagarjuna (150-250 CE)**: A Buddhist philosopher and founder of Mahayana Buddhism. He developed the concept of "emptiness" (shunyata), which

KeyboardInterrupt: 

In [30]:

def get_n_words_from_text(text:str, number_of_words=1000):
    words = text.split()
    first_n_words = ' '.join(words[:number_of_words])
    return first_n_words

# with open('../../data/buddhist-texts/majjhima nikaya.txt', 'r') as file:
with open('C:\\shared-drive\\dev\\python-playground\\data\\buddhist-texts\\majjhima nikaya.txt', 'r', encoding='utf-8') as file:
    large_text = file.read()
    
large_text = get_n_words_from_text(large_text, number_of_words=19000)    
stream_response(f"""
Using the text below, which is surrounded by <text> tags, summarize what is being discussed.  Include summaries of each sutta, the key points discussed, and then provide a summary of all the suttas, highlighting the key concepts and terms
<text>
{large_text}
</text>
""")    

ReadError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [38]:
import openai

client = openai.OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "THIS MUST BE 127.0.0.1, not localhost
    api_key = "sk-no-key-required"
)

def stream_response(question):
    system_message = '''
You are an expert at communicating in only the json format to help determine which function should be called, based on a user's question.  
You do not use preamble, explanation, or pleasantries.  You only respond with json strings, no matter what is asked of you. 
If you are unable to respond in json, you simply respond with ''.
You are provided an OpenAPI spec json object describing the functions available for you to call (aka tools), based on the user's question.

Examples for you to follow for chain of thought reasoning
Example 1:
This is the OpenAI (not openapi) spec json object describing the functions available to you:
{
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": [
                "celsius",
                "fahrenheit"
              ]
            }
          },
          "required": [
            "location"
          ]
        }
      }
    }
  ]
}

Example question: How is the whether?
Example function call response: Your response should be this json object:
{
   "choices": [
        {
          "index": 0,
          "message": {
            "role": "assistant",
            "content": null,
            "tool_calls": [
              {
                "id": "call_abc123",
                "type": "function",
                "function": {
                  "name": "get_current_weather",
                  "arguments": "{"location": "Boston, MA"}"
                }
              }
            ]
          },
          "finish_reason": "tool_calls"
        }
    ],
} 
    '''
    completion = client.chat.completions.create(
        model="llama-3",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question}
        ],
        stream=True,  # Whether to stream the response or return the full response at once
        max_tokens=8000,  # The maximum number of tokens to generate in the completion
        temperature=0.5,  # The temperature of the model, controlling the randomness of the output
        top_p=1.0,  # The nucleus sampling parameter, controlling the diversity of the output
        n=1,  # The number of completions to generate
        # stop=None,  # A list of stop sequences to stop the completion at
        stop=None,
        presence_penalty=0.0,  # The penalty for generating tokens that appear in the prompt
        frequency_penalty=0.0,  # The penalty for generating tokens that appear frequently in the prompt
        logit_bias={},  # A dictionary of token IDs and associated biases to apply to the logits
    )
    
    # print(completion.choices[0].message)
    for chunk in completion:
        print(chunk.choices[0].delta.content, end="")

stream_response('''
Here are the tools/functions available to you:
{
    "tools": [
        {
            "type": "function",
            "function": {
                "name": "get_user_details",
                "parameters": {
                    "type": "object",
                    "title": "get_user_details",
                    "properties": {
                        "first_name": {
                            "title": "First Name",
                            "type": "string"
                        },
                        "last_name": {
                            "title": "Last Name",
                            "type": "string" #integer
                        }
                    },
                    "required": [ "first_name", "last_name" ]
                }
            }
        }
  ]
}       

Using the above, provide either a function call  response if the function seems like it can answer the question, or simply an empty string.

Question: who is bob?
''')

{
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": null,
                "tool_calls": [
                    {
                        "id": "call_abc123",
                        "type": "function",
                        "function": {
                            "name": "get_user_details",
                            "arguments": "{\"first_name\": \"Bob\", \"last_name\": \"\"}"
                        }
                    }
                ]
            },
            "finish_reason": "tool_calls"
        }
    ]
}None